In [78]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen, urlretrieve

import pandas as pd
import re
import time



In [56]:

url = r"https://www.autotrader.co.uk/van-search?postcode=OX4%202FU&make=Ford&model=Transit&body-type=Panel%20Van&supplied-price-to=5000&include-delivery-option=on&advertising-location=at_vans&page=1"

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

soup = BeautifulSoup(webpage, "lxml")


soup.find("li", {"class":"paginationMini__count"}).get_text().strip()

pages = re.findall("\d+", soup.find("li", {"class":"paginationMini__count"}).get_text().strip())
max_page = max(map(int, pages))

In [70]:
auto_url = "https://www.autotrader.co.uk"
vans_df = []

for i in range(1, max_page):
    url = f"https://www.autotrader.co.uk/van-search?postcode=OX4%202FU&make=Ford&model=Transit&body-type=Panel%20Van&supplied-price-to=5000&include-delivery-option=on&advertising-location=at_vans&page={i}"
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    soup = BeautifulSoup(webpage, "lxml")

    vans = soup.find_all("article", {"class": "product-card"})

    for van in vans:
        temp_dict = {}

        price = van.find("div", {"class": "product-card-pricing__price"}).get_text().strip().replace(",","")

        temp_dict["Price (£)"] = float(re.findall("\d+", price)[0])

        if "VAT" in price.upper():
            temp_dict["VAT"] = True
        else:
            temp_dict["VAT"] = False

        temp_dict["Type"] = van.find("h3", {"class": "product-card-details__title"}).get_text().strip()
        temp_dict["Specs"] = van.find("p", {"class": "product-card-details__subtitle"}).get_text().strip()
        try:
            temp_dict["Headline"] = van.find("p", {"class": "product-card-details__attention-grabber"}).get_text().strip()
        except AttributeError:
            temp_dict["Headline"] = "None"

        key_specs = van.find("ul", {"class": "listing-key-specs"})
        key_specs = key_specs.find_all("li", {"class" :"atc-type-picanto--medium"})
        key_specs = [x.get_text().strip() for x in key_specs]

        for ks in key_specs:
            if "reg" in ks.lower():
                temp_dict["Year"] = int(re.findall("\d+", ks)[0])
            elif "WB" in ks.upper():
                temp_dict["Wheel Base"] = ks.upper()
            elif "miles" in ks.lower():
                temp_dict["Milleage"] = float(re.findall("\d+", ks.replace(",", ""))[0])
            elif ks.endswith("L"):
                temp_dict["Engine"] = float(re.findall("\d+\.\d+", ks)[0])
            elif "BHP" in ks.upper():
                temp_dict["Break Horse Power"] = float(re.findall("\d+", ks)[0])
            elif ks.lower() in ["manual", "automatic"]:
                temp_dict["Transmission"] = ks.lower()
            elif ks.lower() in ["diesel", "petrol", "electric"]:
                temp_dict["Fuel"] = ks.lower()
            elif "seats" in ks.lower():
                temp_dict["Seats"] = int(re.findall("\d+", ks)[0])
            elif "Year" not in temp_dict.keys():
                temp_dict["Year"] = int(re.findall("\d+", ks)[0])


        temp_dict["Dealer"] = van.find("h3", {"class":"product-card-seller-info__name atc-type-picanto"}).get_text().strip()

        location = van.find_all("li", {"class":"product-card-seller-info__spec-item atc-type-picanto"})
        for loc in location:
            for sub_loc in loc.get_text().strip().splitlines():
                if "miles" in sub_loc:
                    temp_dict["Distance"] = float(re.findall("\d+",sub_loc)[0])

        url_van = van.find("a", {"class": "js-click-handler listing-fpa-link tracking-standard-link"})
        temp_dict["url_van"] = f"{auto_url}{url_van['href']}"
        temp_dict["page"] = i
        vans_df.append(temp_dict)

    time.sleep(2)

vans_df = pd.DataFrame(vans_df)

IndexError: list index out of range

In [59]:
vans_df

,Price (£),VAT,Type,Specs,Headline,Year,Wheel Base,Milleage,Engine,Break Horse Power,Transmission,Fuel,Seats,Dealer,Distance,url_van
0,4990.0,False,Ford Transit,2.2 TDCi 260 Duratorq Low Roof Panel Van SWB 5...,WARRANTY-SERVICED-CLEAN-NO VAT,2008.0,SWB,63000.0,2.2,84.0,manual,diesel,3.0,Auto Globe Limited,29.0,https://www.autotrader.co.uk/van-details/20220...
1,3500.0,False,Ford Transit,2.4 TDCi 350 Duratorq High Roof LWB 3dr,"Good runner, Long MOT",2006.0,LWB,81000.0,2.4,98.0,manual,diesel,3.0,Heathrow General Cars Denham,34.0,https://www.autotrader.co.uk/van-details/20220...
2,2500.0,False,Ford Transit,2.4 TD 350 LWB High Roof 3dr,"GOOD VAN, RUNS & DRIVES GREAT",2002.0,LWB,150000.0,2.4,118.0,manual,diesel,3.0,B A Cars,12.0,https://www.autotrader.co.uk/van-details/20220...
3,4990.0,False,Ford Transit,2.4 TDCi 350 Trend Medium Roof Panel Van LWB M...,"Ready to go, NO VAT",2010.0,LWB,84000.0,2.4,113.0,manual,diesel,3.0,St James's Cars,72.0,https://www.autotrader.co.uk/van-details/20220...
4,2894.0,True,Ford Transit,2.2 TDCi 280 Duratorq MWB 5dr (MWB),12 months mot,2010.0,MWB,103000.0,2.2,84.0,manual,diesel,3.0,City Motors Coventry,53.0,https://www.autotrader.co.uk/van-details/20220...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,3950.0,False,Ford Transit,2.2 TDCi 280 Duratorq MWB 5dr (MWB),None,2010.0,MWB,133000.0,2.2,84.0,manual,diesel,3.0,Private seller,61.0,https://www.autotrader.co.uk/van-details/20220...
183,4995.0,False,Ford Transit,"2.2 TDCi 350 LWB High Roof 3dr (Euro 5, LWB)",Welfare unit / mess van,2012.0,LWB,143000.0,2.2,98.0,manual,diesel,3.0,Commercial Van centre,99.0,https://www.autotrader.co.uk/van-details/20220...
184,4795.0,True,Ford Transit,2.2 TDCi 260 Duratorq Low Roof Panel Van SWB 5...,None,2007.0,SWB,58000.0,2.2,84.0,manual,diesel,3.0,Wizz commercials,55.0,https://www.autotrader.co.uk/van-details/20220...
185,2850.0,True,Ford Transit,2.2 TDCi 350 Duratorq Medium Roof Van MWB Medi...,None,2011.0,MWB,210000.0,2.2,138.0,manual,diesel,NaN,BSG Supplies Ltd,87.0,https://www.autotrader.co.uk/van-details/20220...


In [60]:
import json

with open("vans.json", "w") as f:
    json.dump(vans_df.to_json(), f, indent=4)

In [69]:
vans_df[vans_df["Headline"]]

KeyError: False

In [71]:
url = "https://www.autotrader.co.uk/van-details/202205175793957?sort=relevance&radius=1501&body-type=Panel%20Van&make=Ford&postcode=ox42fu&onesearchad=New&onesearchad=Nearly%20New&onesearchad=Used&model=Transit&supplied-price-to=5000&include-delivery-option=on&advertising-location=at_vans&page=4"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

soup = BeautifulSoup(webpage, "lxml")


In [84]:

imgs = soup.find_all("img")

for img in imgs:
    print(img)
    url_img = img["src"]
    if url_img.endswith(".jpg"):
        print(url_img)
        urlretrieve(url_img, "temp.jpg")

<img alt="Image 1" sizes="(max-width: 320px) 300px, (max-width: 480px) 480px, (max-width: 600px) 600px, (max-width: 720px) 720px, 800px" src="https://m.atcdn.co.uk/a/media/w800h600/04477b3b356548569d0cc57bb8b38fe9.jpg" srcset="https://m.atcdn.co.uk/a/media/w300h225/04477b3b356548569d0cc57bb8b38fe9.jpg 320w, https://m.atcdn.co.uk/a/media/w480h360/04477b3b356548569d0cc57bb8b38fe9.jpg 480w, https://m.atcdn.co.uk/a/media/w600h450/04477b3b356548569d0cc57bb8b38fe9.jpg 600w, https://m.atcdn.co.uk/a/media/w720h540/04477b3b356548569d0cc57bb8b38fe9.jpg 720w, https://m.atcdn.co.uk/a/media/w800h600/04477b3b356548569d0cc57bb8b38fe9.jpg 800w"/>
https://m.atcdn.co.uk/a/media/w800h600/04477b3b356548569d0cc57bb8b38fe9.jpg
<img alt="Seller Logo" class="sc-khIimk lkXqVR" data-gui="dealer-logo" src="https://dealerlogo.atcdn.co.uk/at2/adbranding/2514804/images/fpa_logo.gif" width="300"/>
